In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os,sys
for dirname, _, filenames in os.walk('/kaggle/input/d/alexeychinyaev/src-modules'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

module_path = os.path.abspath(os.path.join(os.pardir))# + '/input/src-modules/'
print(f'module path = {module_path}')
if module_path not in sys.path:
    sys.path.append(module_path)

#print(f'module path = {sys.path}')
    
from scipy.sparse import csr_matrix, coo_matrix    
from metrics import precision_at_k, recall_at_k
from recommenders import MainRecommender
from utils import prefilter_items
import lightfm.evaluation
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../input/hw5-data/transaction_data.csv')
item_features = pd.read_csv('../input/hw5-data/product.csv')
user_features = pd.read_csv('../input/hw5-data/hh_demographic.csv')

In [34]:
data.describe().style.set_precision(0)

In [4]:
data.info(),'--------------------', data.shape

In [5]:
item_features.info(),'--------------------', item_features.describe(),'--------------------',  item_features.shape

In [6]:
item_features.info()

In [8]:
user_features.describe()

In [9]:
user_features.shape

In [10]:
#переименовываем заголовки столбцов, для удобства работы, все строчные
item_features.columns = [col.lower() for col in item_features.columns]
data.columns = [col.lower() for col in data.columns]

# переименовываем заголовки для user и item
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
data.rename(columns={'household_key': 'user_id'}, inplace=True)
data.rename(columns={'product_id': 'item_id'}, inplace=True)

In [11]:
%%time
data = prefilter_items(data)

In [12]:
data

In [13]:
%%time
model = MainRecommender(data)

In [14]:
user = 2000

In [15]:
user_purchases = data[(data['user_id'] == user)&(data['item_id'] != 999999)]['item_id']
user_purchases = np.array(user_purchases)
len(user_purchases)

In [16]:
user_purchases

In [17]:
items = model.get_similar_items_recommendation(user, N=len(user_purchases))
items[:5]

### 2. Работа с признаками

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 17%  

У Вас, скорее всего, возникнет проблема со временем обучения. Почему они возникает?    

Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

In [18]:
data.info()

In [19]:
data.describe().style.set_precision(1)

In [20]:
data.nunique()

In [21]:
for col in data.columns:
    print(col, ':', data[col].unique(), '\n')

In [22]:
item_features.nunique()

In [23]:
for col in item_features.columns:
    print(col, ':', item_features[col].unique(), '\n')

In [27]:
#Первый год
data[data['week_no']<=52].hist(figsize=(16,16), bins=100, grid=True);

In [28]:
#второй год
data[data['week_no']>52].hist(figsize=(16,16), bins=100, grid=True);

In [ ]:
list = data.columns
sns.kdeplot(data=data[list[1]])

In [ ]:
sns.kdeplot(data=data[list[1]],fill=True)


In [ ]:
sns.kdeplot(data=data[list[2]],fill=True)

In [29]:
sns.kdeplot(data=data['week_no'],fill=True)

In [30]:
sns.kdeplot(data=data['price'],fill=True)

In [46]:
sns.kdeplot(data=data['trans_time'],fill=True)

In [45]:
data.describe().style.set_precision(0)

Больше всего покупок совершалось в районе 18-43

Популярный магазин № 424

Виден рост кол-во покупок в начале, далее рост стабилизируется.
больше всего покупок на 558 день,это 193 день второго года от начала отсчета. Если считать, что отсчет велся от 1 января, то 193 день это 11 июля. Странно что 11 июля совершенно больше всего покупок, обычно больше всего покупок совершается в дни распродаж в конце года, то можно считать что отсчет в датасете велся не с 1 января и привязываться к датам не стоит.


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [56]:
f, ax = plt.subplots(figsize=(7, 6))
ax.set_yscale("log")
sns.histplot(data=data['item_id'])

In [ ]:
f, ax = plt.subplots(figsize=(7, 6))
ax.set_yscale("log")

sns.histplot(data=data['quantity'])

**3.** LightFM 

In [58]:
data = pd.read_csv('../input/hw5-data/transaction_data.csv')
item_features = pd.read_csv('../input/hw5-data/product.csv')
user_features = pd.read_csv('../input/hw5-data/hh_demographic.csv')

In [60]:
data.shape